Test code to add incomplete models to a GTF

In [1]:
import CGAT.GTF as GTF
import CGAT.IOTools as IOTools

import copy 
import numpy as np

import collections

In [32]:
class options_attr():
    pass

options = options_attr
options.infile_gtf = '../../run/annotations/sequins.gtf.gz'
options.outfile = 'test.gtf.gz'

In [44]:
ninput = 0
nshort = 0

options.fraction = 0.01

transcript2exons = collections.defaultdict(set)

gtfs = GTF.iterator(IOTools.openFile(options.infile_gtf))

for gtf_lines in GTF.transcript_iterator(gtfs, strict=0):

    if gtf_lines[0].feature == 'exon':
        ninput += 1

        transcript_id = gtf_lines[0].transcript_id

        if len(gtf_lines)>4:
            exon_number = 1
            for exon in gtf_lines:
                transcript2exons[transcript_id].add(exon_number)
                exon_number += 1
        else:
            nshort += 1


In [84]:
# list with each transcript repeated for each available exon
flat_transcripts = [t_id for t_id in transcript2exons.keys()
                    for exon in list(transcript2exons[t_id])[:-2]]

try:
    skip_transcripts = list(
        np.random.choice(flat_transcripts, int(ninput * options.fraction),
                         replace=False))
except ValueError as e:
    raise ValueError('cannot ask for more skip exon additional '
    'models than there are exons available: available exons=%i, '
    'additional_models=%i.   numpy error: %s' % (
        len(flat_transcripts),
        int(ninput * options.fraction),
        e))


In [87]:
print(list(transcript2exons[skip_transcripts[0]])[:-2])
print(list(transcript2exons[skip_transcripts[0]]))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [88]:
print(skip_transcripts)

['R1_33_1']


In [90]:
noutput = 0


with IOTools.openFile(options.outfile, "w") as outf:
    gtfs = GTF.iterator(IOTools.openFile(options.infile_gtf))

    for gtf_lines in GTF.transcript_iterator(gtfs, strict=0):

        if gtf_lines[0].feature == 'exon':

            # write out every transcript_id already in gtf
            noutput += 1

            for gtf in gtf_lines:
                outf.write("%s\n" % str(gtf))

            # if transcript has been selected, remove an exon at
            # random and output as a new transcript
            transcript_id = gtf_lines[0].transcript_id

            if transcript_id in skip_transcripts:

                start_exons = np.random.choice(
                    list(transcript2exons[transcript_id])[:-2], 
                    skip_transcripts.count(transcript_id),
                    replace=False)

                for start in start_exons:
                    
                    end = np.random.choice(
                        list(transcript2exons[transcript_id])[start+1:])
                    print(start)
                    print(list(transcript2exons[transcript_id])[start+1:])
                    print(start, end)
                    
                    noutput += 1

                    transcript_id = gtf_lines[0].transcript_id

                    new_transcript_id = "%s_incomplete_%i_%i" % (transcript_id, start, end)

                    for ix, gtf in enumerate(gtf_lines):

                        # we only want to include the eoxns between start and end
                        if (ix + 1) >= start and (ix+1) <= end:

                            gtf2 = copy.copy(gtf)
                            gtf2.setAttribute('transcript_id', new_transcript_id)

                            outf.write("%s\n" % str(gtf2))


15
[17, 18, 19, 20, 21, 22, 23]
15 19
